In [88]:
import requests
import pandas as pd
import pprint
import ssl
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import os
import time
from requests.exceptions import ReadTimeout

In [11]:
# importando bibliotecas conexão PDGT
import json
import argparse
import subprocess
import boto3
import time
from pyathena import connect
import pandas.io.sql as sqlio
import sys
from ydata_profiling import ProfileReport
from botocore import UNSIGNED
from botocore.config import Config
import boto3.session
from botocore import exceptions
from scipy.stats import chi2_contingency

In [12]:
class CustomException(Exception):
    pass

In [13]:
json_manifest_dbt = "target/manifest.json"
athena_bucket = "todos-athena-us-east-1"
athena_tmp_folder = f"s3://{athena_bucket}/"

In [14]:
def execute_athena_sql(query):
    client = boto3.client('athena', region_name='us-east-1')
    queryStart = client.start_query_execution(
    QueryString = query,
    ResultConfiguration = { 'OutputLocation': athena_tmp_folder})
    queryExecution = client.get_query_execution(QueryExecutionId=queryStart['QueryExecutionId'])
    while queryExecution['QueryExecution']['Status']['State'] in ('RUNNING', 'QUEUED'):
        time.sleep(5)
        queryExecution = client.get_query_execution(QueryExecutionId=queryStart['QueryExecutionId'])

In [15]:
def execute_athena_query(query):
    cursor = connect(s3_staging_dir=athena_tmp_folder,
                    region_name="us-east-1").cursor()
    cursor.execute(query)
    colls=','.join(str(f"{e[0]}") for e in cursor.description)
    results=pd.DataFrame(list(cursor), columns=colls.split(","))
    return results

In [16]:
url = "https://servicodados.ibge.gov.br/api/v1/pesquisas/23" 

#teste 1
#https://servicodados.ibge.gov.br/api/v3/agregados/5436/periodos/202302/variaveis/5932?localidades=N6[all]&classificacao=2[all]

#teste 2
#https://servicodados.ibge.gov.br/api/v3/agregados/1428/periodos/2010/variaveis/1188|1189?localidades=N6[all]&classificacao=2[all]|1[6795]

#pesquisas
#https://servicodados.ibge.gov.br/api/v1/pesquisas

In [17]:
class TLSAdapter(requests.adapters.HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("DEFAULT@SECLEVEL=1")
        ctx.options |= 0x4   # <-- the key part here, OP_LEGACY_SERVER_CONNECT
        kwargs["ssl_context"] = ctx
        return super(TLSAdapter, self).init_poolmanager(*args, **kwargs)

In [18]:
with requests.session() as s:
    s.mount("https://", TLSAdapter())
    response = s.get(url)
    data = response.json()

In [19]:
#df = pd.DataFrame(data)
#df

In [20]:
#df.to_excel('pesquisas_ibge.xlsx', index=False, sheet_name='pesquisas')

In [21]:
pprint.pprint(data)

{'contexto': '101010',
 'descricao': None,
 'id': 23,
 'nome': 'Censo 2010',
 'observacao': None,
 'periodos': [{'fonte': ['IBGE, Censo Demográfico 2010'],
               'nota': [],
               'periodo': '2010',
               'publicacao': '26/09/2016 12:44:19',
               'versao': 23}]}


In [22]:
#for i in range(0, len(informacoes)):
#    currentItem = informacoes[i]
#    df.loc[i] = [data[i]['']]

In [23]:
#teste = pd.DataFrame(informacoes.features.tolist())
#pd.json_normalize(teste).head()

In [71]:
query_endereco = """
--BASE PACIENTES
--SELECT count(*) from (
WITH agg_pacientes AS (
    SELECT
        ag.id_paciente,
        ag.cpf,
        MIN(ag.dt_agendamento) AS min_dt_ag,
        MAX(ag.dt_agendamento) AS max_dt_ag,
        DATE_DIFF('year', MIN(ag.dt_agendamento), current_date) AS anos_utilizacao,
        DATE_DIFF('month', MIN(ag.dt_agendamento), current_date) AS meses_utilizacao,
        DATE_DIFF('day', MAX(ag.dt_agendamento), current_date) AS dias_ult_utilizacao,
        COALESCE(utlz.qtd_utilizacao, 0) AS qtd_utilizacao,
        COALESCE(cst.qtd_consultas, 0) AS qtd_consultas,
        --COALESCE(vexpc.qtd_exam_proc, 0) AS qtd_exam_proc,
        COALESCE(oe.qtd_orc_executado, 0) AS qtd_exam_proc,
        COALESCE(ptcl.qtd_particular, 0) AS qtd_particular,
        COALESCE(cst.tt_consulta, 0) AS tt_consulta,
        COALESCE(cst.tm_consulta, 0) AS tm_consulta,
        --COALESCE(expc.tt_exam_proc, 0) AS tt_exam_proc,
        COALESCE(oe.tt_orc_executado, 0) AS tt_exam_proc,
        COALESCE(one.qtd_orc_nao_executado, 0) AS qtd_orc_nao_executado,
        COALESCE(one.tt_orc_nao_executado, 0) AS tt_orc_nao_executado,
        CASE WHEN COALESCE(cst.qtd_consultas, 0) <> 0 THEN (COALESCE(cst.tt_consulta, 0) + COALESCE(oe.tt_orc_executado, 0)) / cst.qtd_consultas
             ELSE NULL END AS tm_utilizacao
    FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
    LEFT JOIN (
        SELECT
            ag.id_paciente,
            COUNT(*) AS qtd_consultas,
            SUM(ag.valor) AS tt_consulta,
            AVG(ag.valor) AS tm_consulta
        FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
        WHERE 1=1
            AND ag.id_grupoprocedimento IN (1, 8) --'Bonificação', 'Retorno'?
            AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
        GROUP BY
            ag.id_paciente
    ) cst ON cst.id_paciente = ag.id_paciente
    LEFT JOIN (
        SELECT
            ag.id_paciente,
            COUNT(*) AS qtd_exam_proc,
            SUM(ag.valor) AS tt_exam_proc
        FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
        WHERE 1=1
            AND ag.id_grupoprocedimento IN (5, 4) --'Exames Laboratoriais' não tem essa classificação
            AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
        GROUP BY
            ag.id_paciente
    ) expc ON expc.id_paciente = ag.id_paciente
    LEFT JOIN (
        SELECT
        pp.pacienteid AS id_paciente,
        COUNT(*) AS qtd_orc_executado,
        SUM(pp.valor) AS tt_orc_executado
    FROM todos_data_lake_trusted_feegow.propostas pp
    WHERE 1=1
        AND pp.staid IN (5, 2)
    GROUP BY
        pp.pacienteid
	) oe ON oe.id_paciente = ag.id_paciente
    LEFT JOIN (
        SELECT
            pp.pacienteid AS id_paciente,
            COUNT(*) AS qtd_orc_nao_executado,
            SUM(pp.valor) AS tt_orc_nao_executado
        FROM todos_data_lake_trusted_feegow.propostas pp
        WHERE 1=1
            AND pp.staid NOT IN (5, 2)
        GROUP BY
            pp.pacienteid
    ) one ON one.id_paciente = ag.id_paciente
    LEFT JOIN (
        SELECT
            ag.id_paciente,
            COUNT(*) AS qtd_particular
        FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
        WHERE 1=1
            AND ag.id_grupoprocedimento IN (1, 8)
            AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
            AND ag.nm_tabela = 'PARTICULAR*'
        GROUP BY
            ag.id_paciente
    ) ptcl ON ptcl.id_paciente = ag.id_paciente
    LEFT JOIN (
        SELECT
            base.id_paciente,
            SUM(base.visitas) AS qtd_utilizacao
        FROM (
            SELECT
                ag.id_paciente,
                ag.dt_agendamento,
                COUNT(*) AS agregador,
                1 AS visitas
            FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
            WHERE 1=1
                AND ag.id_grupoprocedimento IN (1, 8, 5, 4) --'Exames Laboratoriais',
                AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
            GROUP BY
                ag.id_paciente, ag.dt_agendamento
        ) base
        GROUP BY
            base.id_paciente
    ) utlz ON utlz.id_paciente = ag.id_paciente
    LEFT JOIN (
        SELECT
            base.id_paciente,
            SUM(base.visitas) AS qtd_exam_proc
        FROM (
            SELECT
                ag.id_paciente,
                ag.dt_agendamento,
                COUNT(*) AS agregador,
                1 AS visitas
            FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
            WHERE 1=1
                AND ag.id_grupoprocedimento IN (5, 4) --'Exames Laboratoriais' não tem na agendamentos?
                AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
            GROUP BY
                ag.id_paciente, ag.dt_agendamento
        ) base
        GROUP BY
            base.id_paciente
    ) vexpc ON vexpc.id_paciente = ag.id_paciente
    WHERE 1=1
    AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
    GROUP BY
        ag.id_paciente,
        ag.cpf,
        utlz.qtd_utilizacao,
        cst.qtd_consultas,
        cst.tm_consulta,
        vexpc.qtd_exam_proc,
        expc.tt_exam_proc,
        one.qtd_orc_nao_executado,
        one.tt_orc_nao_executado,
        ptcl.qtd_particular,
        cst.tt_consulta,
        oe.tt_orc_executado,
        oe.qtd_orc_executado
),
canal_mais_utilizado AS (
    WITH canal_paciente AS (
        SELECT
            ag.id_paciente,
            COALESCE(ag.nm_canal, 'Não informado') nm_canal,
            COUNT(*) AS qtd_por_canal
        FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
        WHERE 1=1
        GROUP BY
            ag.id_paciente,
            ag.nm_canal
    ),
    max_canal_paciente AS (
        SELECT
            cp.id_paciente,
            MAX(cp.qtd_por_canal) AS max_qtd_canal
        FROM canal_paciente cp
        GROUP BY cp.id_paciente
    )
    SELECT DISTINCT
        cp.id_paciente,
        cp.nm_canal
    FROM canal_paciente cp
    LEFT JOIN max_canal_paciente mcp ON cp.id_paciente = mcp.id_paciente
    WHERE 1=1
    AND cp.qtd_por_canal = mcp.max_qtd_canal
),
regional_mais_utilizada AS (
        WITH regional_paciente AS (
        SELECT
            ag.id_paciente,
            ag.regional,
            COUNT(*) AS qtd_por_regional
        FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
        WHERE 1=1
        AND ag.regional IS NOT NULL
        AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
        GROUP BY
            ag.id_paciente,
            ag.regional
    ),
    max_regional_paciente AS (
        SELECT
            rp.id_paciente,
            MAX(rp.qtd_por_regional) AS max_qtd_regional
        FROM regional_paciente rp
        GROUP BY rp.id_paciente
    )
    SELECT DISTINCT
        rp.id_paciente,
        rp.regional
    FROM regional_paciente rp
    LEFT JOIN max_regional_paciente mrp ON rp.id_paciente = mrp.id_paciente
    WHERE 1=1
    AND rp.qtd_por_regional = mrp.max_qtd_regional
),
tabela_consulta AS (
        SELECT
        ag.id_paciente,
        ag.dt_agendamento,
        CASE WHEN ag.nm_tabela = 'Cartão de TODOS*' THEN 'Cartão de TODOS'
             WHEN ag.nm_tabela = 'PARTICULAR*' THEN 'Particular'
             ELSE 'Outros' END AS tabela_particular
    FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
    WHERE 1=1
    AND ag.id_grupoprocedimento IN (1, 8) --Consultas, Sessão
    AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
),
especialidade_mais_utilizada AS (
        WITH especialidade_paciente AS (
        SELECT
            ag.id_paciente,
            ag.nm_especialidade AS especialidade,
            COUNT(*) AS qtd_por_especialidade
        FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
        WHERE 1=1
        AND ag.nm_especialidade IS NOT NULL
        AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
        GROUP BY
            ag.id_paciente,
            ag.nm_especialidade
    ),
    ranked_especialidade_paciente AS (
        SELECT
            ep.id_paciente,
            ep.especialidade,
            ep.qtd_por_especialidade,
            ROW_NUMBER() OVER(PARTITION BY ep.id_paciente ORDER BY ep.qtd_por_especialidade DESC) AS row_num
        FROM especialidade_paciente ep
    ),
    filtered_especialidade_paciente AS (
        SELECT
            id_paciente,
            especialidade
        FROM ranked_especialidade_paciente
        WHERE row_num = 1
    )
    SELECT DISTINCT
        id_paciente,
        especialidade
    FROM filtered_especialidade_paciente
    ORDER BY id_paciente
),
tabela_mais_utilizada AS (
        WITH tabela_paciente AS (
        SELECT
            ag.id_paciente,
            CASE WHEN ag.nm_tabela = 'Cartão de TODOS*' THEN 'Cartão de TODOS'
             WHEN ag.nm_tabela = 'PARTICULAR*' THEN 'Particular'
             ELSE 'Outros' END AS tabela,
            COUNT(*) AS qtd_por_tabela
        FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
        WHERE 1=1
        AND ag.nm_tabela IS NOT NULL
        AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
        GROUP BY
            ag.id_paciente,
            ag.nm_tabela
    ),
    ranked_tabela_paciente AS (
        SELECT
            ep.id_paciente,
            ep.tabela,
            ep.qtd_por_tabela,
            ROW_NUMBER() OVER(PARTITION BY ep.id_paciente ORDER BY ep.qtd_por_tabela DESC) AS row_num
        FROM tabela_paciente ep
    ),
    filtered_tabela_paciente AS (
        SELECT
            id_paciente,
            tabela
        FROM ranked_tabela_paciente
        WHERE row_num = 1
    )
    SELECT DISTINCT
        id_paciente,
        tabela
    FROM filtered_tabela_paciente
    ORDER BY id_paciente
),
--FILIADO TITULAR
--select count(*) from (
filiado_titular AS (
	WITH crianca AS (
	SELECT
			fl.matricula,
			count(*) dep_crianca,
		avg(fl.idade) idade_media_crianca
	FROM
		pdgt_cartaodetodos_filiado.fl_filiado fl
	WHERE
		1 = 1
		AND fl.flag_titular = 0
		AND fl.idade BETWEEN 0 AND 18
	GROUP BY 
		fl.matricula),
	jovem_adulto AS (
	SELECT
			fl.matricula,
			count(*) dep_jovem_adulto,
		avg(fl.idade) idade_media_jovem_adulto
	FROM
		pdgt_cartaodetodos_filiado.fl_filiado fl
	WHERE
		1 = 1
		AND fl.flag_titular = 0
		AND fl.idade BETWEEN 19 AND 28
	GROUP BY 
		fl.matricula),
	adulto AS (
	SELECT
			fl.matricula,
			count(*) dep_adulto,
		avg(fl.idade) idade_media_adulto
	FROM
		pdgt_cartaodetodos_filiado.fl_filiado fl
	WHERE
		1 = 1
		AND fl.flag_titular = 0
		AND fl.idade BETWEEN 29 AND 48
	GROUP BY 
		fl.matricula),
	senior AS (
	SELECT
			fl.matricula,
			count(*) dep_senior,
		avg(fl.idade) idade_media_senior
	FROM
		pdgt_cartaodetodos_filiado.fl_filiado fl
	WHERE
		1 = 1
		AND fl.flag_titular = 0
		AND fl.idade BETWEEN 49 AND 100
	GROUP BY 
		fl.matricula),
	homens AS (
	SELECT
			fl.matricula,
			count(*) dep_homem,
		avg(fl.idade) idade_media_homem
	FROM
		pdgt_cartaodetodos_filiado.fl_filiado fl
	WHERE
		1 = 1
		AND fl.flag_titular = 0
		AND fl.sexo = 'MASCULINO'
	GROUP BY 
		fl.matricula),
	mulheres AS (
	SELECT
			fl.matricula,
			count(*) dep_mulher,
		avg(fl.idade) idade_media_mulher
	FROM
		pdgt_cartaodetodos_filiado.fl_filiado fl
	WHERE
		1 = 1
		AND fl.flag_titular = 0
		AND fl.sexo = 'FEMININO'
	GROUP BY 
		fl.matricula)
	SELECT 
		fl.matricula,
		fl.cpf,
		concat(upper(substring(fl.sexo, 1, 1)),
			   lower(substring(fl.sexo, 2))) sexo_fl,
		concat(upper(substring(fl.estado_civil, 1, 1)),
			   lower(substring(fl.estado_civil, 2))) estado_civil,
			   upper(fl.uf) uf,
		fl. idade idade_fl,
		fl.dependentes,
		COALESCE(cr.dep_crianca,
		0) dep_crianca,
		cr.idade_media_crianca,
		COALESCE(ja.dep_jovem_adulto,
		0) dep_jovem_adulto,
		ja.idade_media_jovem_adulto,
		COALESCE(ad.dep_adulto,
		0) dep_adulto,
		ad.idade_media_adulto,
		COALESCE(sr.dep_senior,
		0) dep_senior,
		sr.idade_media_senior,
		COALESCE(hm.dep_homem,
		0) dep_homem,
		hm.idade_media_homem,
		COALESCE(ml.dep_mulher,
		0) dep_mulher,
		ml.idade_media_mulher,
		fl.flag_desfiliado,
		fl.flag_refiliado,
		fl.status_atual_desc,
		fl.faixa_etaria faixa_etaria_fl,
		fl.dt_filiacao,
		fl.data_ultima_desfiliacao,
		fl.data_ultima_refiliacao,
		CASE
			WHEN fl.status_atual_desc = 'Ativo' THEN date_diff('month',
			fl.dt_filiacao,
			current_date)
			WHEN fl.status_atual_desc = 'Desfiliado' THEN date_diff('month',
			fl.dt_filiacao,
			fl.data_ultima_desfiliacao)
			ELSE NULL
		END AS meses_filiacao,
		upper(fl.prvd_tipo_venda_promotor) prvd_tipo_venda_promotor
	FROM
		pdgt_cartaodetodos_filiado.fl_filiado fl
	LEFT JOIN crianca cr ON
		cr.matricula = fl.matricula
	LEFT JOIN jovem_adulto ja ON
		ja.matricula = fl.matricula
	LEFT JOIN adulto ad ON
		ad.matricula = fl.matricula
	LEFT JOIN senior sr ON
		sr.matricula = fl.matricula
	LEFT JOIN homens hm ON
		hm.matricula = fl.matricula
	LEFT JOIN mulheres ml ON
		ml.matricula = fl.matricula
	WHERE
		1 = 1
		--and cpf = '10042511909' --SC308006519
		--and fl.matricula = 'SC308006519'
		AND fl.flag_titular = 1
		AND fl.cpf NOT IN ('99199299394')
		AND fl.status_atual IN (0, 1)
		--and rand() <= 0.1
	--LIMIT 1
	)
SELECT DISTINCT
    ap.*,
    ft.*,
    minpg.tabela_particular AS primeiro_pg,
    CASE WHEN ap.qtd_consultas = ap.qtd_particular AND ap.qtd_consultas <> 0 THEN 1
         ELSE 0 END AS pac_particular,
    CASE WHEN ap.dias_ult_utilizacao > 365 THEN 1
         WHEN ap.dias_ult_utilizacao BETWEEN 181 AND 365 THEN 2
         WHEN ap.dias_ult_utilizacao BETWEEN 91 AND 180 THEN 3
         WHEN ap.dias_ult_utilizacao BETWEEN 46 AND 90 THEN 4
         WHEN ap.dias_ult_utilizacao <= 45 THEN 5
         ELSE NULL END AS score_r,
    CASE WHEN ap.qtd_consultas = 1 THEN 1
         WHEN ap.qtd_consultas = 2 THEN 2
         WHEN ap.qtd_consultas = 3 THEN 3
         WHEN ap.qtd_consultas BETWEEN 4 AND 5 THEN 4
         WHEN ap.qtd_consultas > 5 THEN 5
         ELSE NULL END AS score_f,
    CASE WHEN ap.tm_utilizacao <= 28 THEN 1
         WHEN ap.tm_utilizacao BETWEEN 28.01 AND 56 THEN 2
         WHEN ap.tm_utilizacao BETWEEN 56.01 AND 84 THEN 3
         WHEN ap.tm_utilizacao BETWEEN 84.01 AND 140 THEN 4
         WHEN ap.tm_utilizacao > 140 THEN 5
         ELSE NULL END AS score_v,
    COALESCE(cmu.nm_canal, 'Não informado') AS canal_mais_utilizado,
    COALESCE(UPPER(spe.estado), 'Não informado') AS estado,
    COALESCE(UPPER(cidade), 'Não informado') AS cidade,
    COALESCE(UPPER(bairro), 'Não informado') AS bairro,
    DATE_DIFF('year', pc.nascimento, current_date) AS idade,
    sx.nomesexo AS sexo,
    rmu.regional,
    emu.especialidade,
    tmu.tabela
FROM agg_pacientes ap
LEFT JOIN canal_mais_utilizado cmu ON cmu.id_paciente = ap.id_paciente
LEFT JOIN regional_mais_utilizada rmu ON rmu.id_paciente = ap.id_paciente
LEFT JOIN especialidade_mais_utilizada emu ON emu.id_paciente = ap.id_paciente
LEFT JOIN tabela_consulta minpg ON minpg.id_paciente = ap.id_paciente AND minpg.dt_agendamento = ap.min_dt_ag
LEFT JOIN tabela_mais_utilizada tmu ON tmu.id_paciente = ap.id_paciente 
LEFT JOIN todos_data_lake_trusted_feegow.paciente_endereco spe ON spe.paciente_id = ap.id_paciente
LEFT JOIN todos_data_lake_trusted_feegow.pacientes pc ON pc.id = ap.id_paciente
LEFT JOIN todos_data_lake_trusted_feegow.sexo sx ON sx.id = pc.sexo
JOIN filiado_titular ft ON ft.cpf = ap.cpf
WHERE 1=1
AND ap.cpf IS NOT NULL
AND pc.nascimento IS NOT NULL
AND DATE_DIFF('year', pc.nascimento, current_date) BETWEEN 0 AND 100
--AND LENGTH(spe.estado) = 2
AND spe.estado IN ('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 
'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO')
AND RANDOM() <= 0.01
--ORDER BY ap.cpf
--LIMIT 1
--)
"""

In [72]:
endereco_df = execute_athena_query(query_endereco)

In [73]:
geolocator = Nominatim(user_agent="my_geocoder")

In [99]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="my_geocoder")

minimum_delay = 1

# Function to calculate the distance between two addresses
def calcular_distancia(row):
    try:
        start_time = time.time()  # Record the start time
        location1 = geolocator.geocode(row['endereco_paciente'])
        end_time = time.time()
        request_time = end_time - start_time

        time.sleep(max(0, minimum_delay - request_time))

        start_time2 = time.time()
        location2 = geolocator.geocode(row['endereco_unidade'])
        end_time2 = time.time()  # Record the end time
        request_time2 = end_time2 - start_time2

        time.sleep(max(0, minimum_delay - request_time2))

        if location1 and location2:
            coords1 = (location1.latitude, location1.longitude)
            coords2 = (location2.latitude, location2.longitude)

            return geodesic(coords1, coords2).kilometers
        
        else:
            # If either location is None, return None to skip the row
            return None
        
  #  except ReadTimeout:
   #     # Handle ReadTimeoutError by retrying with exponential backoff
   #     print("Timeout error occurred. Retrying...")
   #     time.sleep(2 ** retries)  # Exponential backoff
   #     return calcular_distancia(row)  # Retry the function
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Apply the function to calculate the distance and create a 'Distancia' column
endereco_df['distancia'] = endereco_df.apply(calcular_distancia, axis=1)

# Examine the DataFrame with the distance column
endereco_df

An error occurred: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=A%2C+0%2C+A%2C+PR&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


,id_paciente,id_unidade,unidade,endereco_paciente,endereco_unidade,cidade_clinica,distancia
0,60812784,19551,AmorSaúde Sabará,"RUA TEÓFILO OTONI, 106, SABARA, MG","AVENIDA EXPEDICIONÁRIO ROMEU JERÔNIMO DANTAS, ...",Sabará,3.426047
1,11777605,19319,AmorSaúde Anápolis,"RUA A, S/N, ANÁPOLIS, GO","AVENIDA GOIÁS, 181, ANÁPOLIS, GO",Anápolis,NaN
2,12685897,19446,AmorSaúde Ribeirão das Neves,"RUA DOUTOR JOAQUIM SOARES LELIS, 415, RIBEIRÃO...","RUA JOÃO CARVALHO FILHO, 28, RIBEIRÃO DAS NEVE...",Ribeirão Das Neves,NaN
3,17527390,19306,AmorSaúde Cascavel,"RUA ANTONIO PEXE, 1554, CASCAVEL, PR","RUA SOUZA NAVES, 3624, CASCAVEL, PR",Cascavel,9.513120
4,16739911,19318,AmorSaúde Goiânia Sul,"RUA H, 0, NAZÁRIO, GO","AVENIDA CIRCULAR, 755, GOIÂNIA, GO",Goiânia,62.057278
...,...,...,...,...,...,...,...
95,60074508,19411,AmorSaúde Bauru,"RUA JOÃO GOMES FERNANDES, 1162, BAURU, SP","RUA GUSTAVO MACIEL, 1680, BAURU, SP",Bauru,5.750459
96,11412714,19388,CMOC (Campinas Sudoeste),"RUA ELIAS FARAH, 21, CAMPINAS, SP","AVENIDA JACAÚNA, 1364, CAMPINAS, SP",Campinas,2.672170
97,57178086,19476,AmorSaúde Mogi das Cruzes,"RUA JEAN BAPTISTE DEBRET, 71, MOGI DAS CRUZES, SP","RUA BASÍLIO BATALHA, 149, MOGI DAS CRUZES, SP",Mogi Das Cruzes,2.607564
98,12905173,19458,AmorSaúde Cariacica,"AVENIDA GETÚLIO VARGAS, 88, CARIACICA, ES","RUA DOM PEDRO II, 1245, CARIACICA, ES",Cariacica,2.869468


In [97]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="my_geocoder")

# Define a minimum delay between requests (in seconds)
minimum_delay = 1.0

# Function to calculate the distance between two addresses
def calcular_distancia(row):
    try:
        start_time = time.time()  # Record the start time
        location1 = geolocator.geocode(row['endereco_paciente'])
        location2 = geolocator.geocode(row['endereco_unidade'])
        end_time = time.time()  # Record the end time
        request_time = end_time - start_time

        if location1 and location2:
            coords1 = (location1.latitude, location1.longitude)
            coords2 = (location2.latitude, location2.longitude)
            sleep_time = max(0, minimum_delay - request_time)
            time.sleep(sleep_time)

            return geodesic(coords1, coords2).kilometers
        else:
            # If either location is None, return None to skip the row
            return None
    except ReadTimeout:
        # Handle ReadTimeoutError by retrying with exponential backoff
        print("Timeout error occurred. Retrying...")
        time.sleep(2)  # Exponential backoff
        return calcular_distancia(row)  # Retry the function
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Apply the function to calculate the distance and create a 'Distancia' column
endereco_df['distancia'] = endereco_df.apply(calcular_distancia, axis=1)

# Examine the DataFrame with the distance column
endereco_df

An error occurred: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=A%2C+0%2C+A%2C+PR&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


,id_paciente,id_unidade,unidade,endereco_paciente,endereco_unidade,cidade_clinica,distancia
0,60812784,19551,AmorSaúde Sabará,"RUA TEÓFILO OTONI, 106, SABARA, MG","AVENIDA EXPEDICIONÁRIO ROMEU JERÔNIMO DANTAS, ...",Sabará,3.426047
1,11777605,19319,AmorSaúde Anápolis,"RUA A, S/N, ANÁPOLIS, GO","AVENIDA GOIÁS, 181, ANÁPOLIS, GO",Anápolis,NaN
2,12685897,19446,AmorSaúde Ribeirão das Neves,"RUA DOUTOR JOAQUIM SOARES LELIS, 415, RIBEIRÃO...","RUA JOÃO CARVALHO FILHO, 28, RIBEIRÃO DAS NEVE...",Ribeirão Das Neves,NaN
3,17527390,19306,AmorSaúde Cascavel,"RUA ANTONIO PEXE, 1554, CASCAVEL, PR","RUA SOUZA NAVES, 3624, CASCAVEL, PR",Cascavel,9.513120
4,16739911,19318,AmorSaúde Goiânia Sul,"RUA H, 0, NAZÁRIO, GO","AVENIDA CIRCULAR, 755, GOIÂNIA, GO",Goiânia,62.057278
...,...,...,...,...,...,...,...
95,60074508,19411,AmorSaúde Bauru,"RUA JOÃO GOMES FERNANDES, 1162, BAURU, SP","RUA GUSTAVO MACIEL, 1680, BAURU, SP",Bauru,5.750459
96,11412714,19388,CMOC (Campinas Sudoeste),"RUA ELIAS FARAH, 21, CAMPINAS, SP","AVENIDA JACAÚNA, 1364, CAMPINAS, SP",Campinas,2.672170
97,57178086,19476,AmorSaúde Mogi das Cruzes,"RUA JEAN BAPTISTE DEBRET, 71, MOGI DAS CRUZES, SP","RUA BASÍLIO BATALHA, 149, MOGI DAS CRUZES, SP",Mogi Das Cruzes,2.607564
98,12905173,19458,AmorSaúde Cariacica,"AVENIDA GETÚLIO VARGAS, 88, CARIACICA, ES","RUA DOM PEDRO II, 1245, CARIACICA, ES",Cariacica,2.869468


In [96]:
# Function to calculate the distance between two addresses
def calcular_distancia(row):
    try:
        location1 = geolocator.geocode(row['endereco_paciente'])
        location2 = geolocator.geocode(row['endereco_unidade'])

        if location1 and location2:
            coords1 = (location1.latitude, location1.longitude)
            coords2 = (location2.latitude, location2.longitude)

            # Sleep for the calculated time
            time.sleep(1)

            return geodesic(coords1, coords2).kilometers
        else:
            # If either location is None, return None to skip the row
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Apply the function to calculate the distance and create a 'Distancia' column
endereco_df['distancia'] = endereco_df.apply(calcular_distancia, axis=1)

# Examine the DataFrame with the distance column
endereco_df

An error occurred: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=A%2C+0%2C+A%2C+PR&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


,id_paciente,id_unidade,unidade,endereco_paciente,endereco_unidade,cidade_clinica,distancia
0,60812784,19551,AmorSaúde Sabará,"RUA TEÓFILO OTONI, 106, SABARA, MG","AVENIDA EXPEDICIONÁRIO ROMEU JERÔNIMO DANTAS, ...",Sabará,3.426047
1,11777605,19319,AmorSaúde Anápolis,"RUA A, S/N, ANÁPOLIS, GO","AVENIDA GOIÁS, 181, ANÁPOLIS, GO",Anápolis,NaN
2,12685897,19446,AmorSaúde Ribeirão das Neves,"RUA DOUTOR JOAQUIM SOARES LELIS, 415, RIBEIRÃO...","RUA JOÃO CARVALHO FILHO, 28, RIBEIRÃO DAS NEVE...",Ribeirão Das Neves,NaN
3,17527390,19306,AmorSaúde Cascavel,"RUA ANTONIO PEXE, 1554, CASCAVEL, PR","RUA SOUZA NAVES, 3624, CASCAVEL, PR",Cascavel,9.513120
4,16739911,19318,AmorSaúde Goiânia Sul,"RUA H, 0, NAZÁRIO, GO","AVENIDA CIRCULAR, 755, GOIÂNIA, GO",Goiânia,62.057278
...,...,...,...,...,...,...,...
95,60074508,19411,AmorSaúde Bauru,"RUA JOÃO GOMES FERNANDES, 1162, BAURU, SP","RUA GUSTAVO MACIEL, 1680, BAURU, SP",Bauru,5.750459
96,11412714,19388,CMOC (Campinas Sudoeste),"RUA ELIAS FARAH, 21, CAMPINAS, SP","AVENIDA JACAÚNA, 1364, CAMPINAS, SP",Campinas,2.672170
97,57178086,19476,AmorSaúde Mogi das Cruzes,"RUA JEAN BAPTISTE DEBRET, 71, MOGI DAS CRUZES, SP","RUA BASÍLIO BATALHA, 149, MOGI DAS CRUZES, SP",Mogi Das Cruzes,2.607564
98,12905173,19458,AmorSaúde Cariacica,"AVENIDA GETÚLIO VARGAS, 88, CARIACICA, ES","RUA DOM PEDRO II, 1245, CARIACICA, ES",Cariacica,2.869468


In [75]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="my_geocoder")

# Function to calculate the distance between two addresses
def calcular_distancia(row):
    try:
        location1 = geolocator.geocode(row['endereco_paciente'])
        location2 = geolocator.geocode(row['endereco_unidade'])

        if location1 and location2:
            coords1 = (location1.latitude, location1.longitude)
            coords2 = (location2.latitude, location2.longitude)
            return geodesic(coords1, coords2).kilometers
        else:
            # If either location is None, return None to skip the row
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Apply the function to calculate the distance and create a 'Distancia' column
endereco_df['distancia'] = endereco_df.apply(calcular_distancia, axis=1)

# Examine the DataFrame with the distance column
endereco_df

An error occurred: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=A%2C+0%2C+A%2C+PR&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


,id_paciente,id_unidade,unidade,endereco_paciente,endereco_unidade,cidade_clinica,distancia
0,60812784,19551,AmorSaúde Sabará,"RUA TEÓFILO OTONI, 106, SABARA, MG","AVENIDA EXPEDICIONÁRIO ROMEU JERÔNIMO DANTAS, ...",Sabará,3.426047
1,11777605,19319,AmorSaúde Anápolis,"RUA A, S/N, ANÁPOLIS, GO","AVENIDA GOIÁS, 181, ANÁPOLIS, GO",Anápolis,NaN
2,12685897,19446,AmorSaúde Ribeirão das Neves,"RUA DOUTOR JOAQUIM SOARES LELIS, 415, RIBEIRÃO...","RUA JOÃO CARVALHO FILHO, 28, RIBEIRÃO DAS NEVE...",Ribeirão Das Neves,NaN
3,17527390,19306,AmorSaúde Cascavel,"RUA ANTONIO PEXE, 1554, CASCAVEL, PR","RUA SOUZA NAVES, 3624, CASCAVEL, PR",Cascavel,9.513120
4,16739911,19318,AmorSaúde Goiânia Sul,"RUA H, 0, NAZÁRIO, GO","AVENIDA CIRCULAR, 755, GOIÂNIA, GO",Goiânia,62.057278
...,...,...,...,...,...,...,...
95,60074508,19411,AmorSaúde Bauru,"RUA JOÃO GOMES FERNANDES, 1162, BAURU, SP","RUA GUSTAVO MACIEL, 1680, BAURU, SP",Bauru,5.750459
96,11412714,19388,CMOC (Campinas Sudoeste),"RUA ELIAS FARAH, 21, CAMPINAS, SP","AVENIDA JACAÚNA, 1364, CAMPINAS, SP",Campinas,2.672170
97,57178086,19476,AmorSaúde Mogi das Cruzes,"RUA JEAN BAPTISTE DEBRET, 71, MOGI DAS CRUZES, SP","RUA BASÍLIO BATALHA, 149, MOGI DAS CRUZES, SP",Mogi Das Cruzes,2.607564
98,12905173,19458,AmorSaúde Cariacica,"AVENIDA GETÚLIO VARGAS, 88, CARIACICA, ES","RUA DOM PEDRO II, 1245, CARIACICA, ES",Cariacica,2.869468


In [76]:
endereco_df.isna().mean()*100

id_paciente           0.0
id_unidade            0.0
unidade               0.0
endereco_paciente     0.0
endereco_unidade      1.0
cidade_clinica        0.0
distancia            42.0
dtype: float64

In [77]:
endereco_df.to_excel('distancia_paciente_validacao.xlsx', index=False, sheet_name='base')

In [86]:
teste = geolocator.geocode('RUA DOUTOR JOAQUIM SOARES LELIS, 415, RIBEIRÃO DAS NEVES, SP')
coords1 = (teste.latitude, teste.longitude)
print(teste.latitude)
print(teste.longitude)

AttributeError: 'NoneType' object has no attribute 'latitude'

In [79]:
teste2 = geolocator.geocode('AVENIDA ESPANHA, 563, ARARAQUARA, SP')
coords2 = (teste2.latitude, teste2.longitude)
print(teste2.latitude)
print(teste2.longitude)
#Paulinho Pipas -> R. Dr. José Santos Neto, 54, Jardim Suzanopolis, Suzano, SP
#paciente 56854683 endereço Rua Dr. Aniz Fadul, 1494, Vila Maluf, Suzano, SP, 08685-020
#

-21.792396
-48.17844


In [80]:
distancia = geodesic(coords1, coords2).kilometers
distancia

3.147726721288721